In [281]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sts
from scipy.special import gamma


In [282]:
def neg_bino_r(k,r,p):
    # To calculate the probability of k .
    # k is the times of success , r is the times of failure , p is the probability of success
    y = gamma(r+k)/(gamma(k+1)*gamma(r)) * (1-p)**r  * p**k
    return y

In [283]:
data = np.load('lda.npz')
data_train = data['train']
data_test = data['test']

data_train = data_train[data_train[:,0]<200]
data_test = data_train[data_train[:,0]<50]

data_train[:,0] = data_train[:,0]-1
data_test[:,0] = data_test[:,0]-1
data_train[:,1] = data_train[:,1]-1
data_test[:,1] = data_test[:,1]-1

voc = max(data_train[:,1])+1
doc = max(data_train[:,0])+1


In [294]:
class PFA():
    def __init__(self,voc,doc,data_train,data_test,eta=0.5,c=1,c0=1,r0=1,gamma=1,alpha=0.05,eps=0.05,k=10):
        #initialization
        #hyperparameter
        self.eta = eta
        self.c = c
        self.c0 = c0
        self.r0 = r0
        self.gamma = gamma
        self.alpha = alpha
        self.eps = eps
        
        #number of document,word type,topic
        self.k = k #主题数量
        self.voc = voc #词汇数
        self.doc = doc #文档数
        
        #gengerative process sample
        self.phi = sts.dirichlet.rvs([self.alpha]*self.voc,size = self.k)  #Topic-word matrix  (voc * k matrix)
        self.pk = sts.beta.rvs(self.c*self.eps,self.c*(1-self.eps),size =self.k) # pk 的分布 1*k matrix
        self.rk = sts.gamma.rvs(self.c0*self.r0,scale=1/self.c0,size=self.k) #rk 1*k matrix
        self.ki = np.array([sts.gamma.rvs(self.rk,scale = self.pk/(1-self.pk)) for i in range(self.doc)]) # doc * k matrix
        
        #sampling count
        self.xik = np.zeros((self.doc,self.k)) #第i个主题第k个topic
        self.xpk = np.zeros((self.voc,self.k)) #第k个主题第p个词的数量
        self.xk = np.zeros((self.k))           #第k个topic的数量
        self.xi = np.zeros((self.doc))         #第i个文档的数量
        
        self.per_plot = []
        
        self.data_test = data_test
        self.data_train = data_train
        
    def sampler(self):
        #1.Count assignment
        for doc_word_count in self.data_train:
            
            doc_index,word_index,word_count = doc_word_count #[0,7,2]
            
            prob =   self.phi[:,word_index] * self.ki[doc_index] # 单词在每个主题的概率 * 该文档每个主题的概率
            prob/=sum(prob) 
            res = sts.multinomial.rvs(word_count,p=prob)
                
            self.xik[doc_index] += res   #第i个文档第k个topic的数量
            self.xpk[word_index] += res          #第k个主题第p个词的数量
            self.xk += res                       #第k个topic的数量
            self.xi[doc_index] += word_count     #第i个文档的数量
        
        #更新topic-doc  matrix,就是我们要替换的部分
        for topic_index in range(self.k):
            self.phi[topic_index] = sts.dirichlet.rvs([self.alpha]*self.voc + model.xpk[:,topic_index])
            
        #更新pk    
        for topic_index in range(self.k):   
            self.pk[topic_index] = sts.beta.rvs(self.c * self.eps + self.xk[topic_index] ,self.c *(1- self.eps)+ self.doc *  self.rk[topic_index] )
        
        #sample rk
        for topic_index in range(self.k):    
            if self.xk[topic_index] == 0: #当xk=0,负二项分布退化为伯努利分布
                self.rk[topic_index] = sts.gamma.rvs(self.c0 * self.r0 , scale = 1/(self.c0 - self.doc * np.log(1- self.pk[topic_index])))
            else:
                self.rk[topic_index] = sts.gamma.rvs(self.c0 * self.r0 + np.sum(self.CRT(topic_index)),scale = 1/(self.c0 - self.doc * np.log(1- self.pk[topic_index])))
        
        # sample self.ki
        for topic_index in range(self.k): 
            self.ki = sts.gamma(self.xik + self.rk,self.pk).rvs()
     
    def compute_perplexity(self):
        phi_theta = np.array([np.sum(self.phi * self.ki[doc_index].reshape(-1,1),0) for doc_index in range(self.doc)])
        phi_theta = phi_theta/np.sum(phi_theta,1).reshape(-1,1)            
        mat = np.zeros((self.doc,self.voc)) 
        for index in self.data_test:#Convert sparse to normal matrix
            mat[index[0],index[1]] = index[-1]    
            per = np.sum(mat * np.log(phi_theta))
            self.per_plot.append(np.exp(- per/np.sum(self.data_test[:,-1])))
        return per
    
    def CRT(self,topic): #CRT flr 
        res = np.array([sum([sts.bernoulli.rvs(self.pk[topic]/(i-1+self.pk[topic])) for i in np.linspace(1,self.xik[d][topic],self.xik[d][topic])]) for d in range(self.doc)])
        return res
    
    
    def demo(self,iteration):
        for it in range(iteration):
            print ("Initial complexity{/d}".)

In [295]:

model = PFA(voc=voc,doc=doc,data_train=data_train,data_test = data_test)


In [ ]:
res = model.perplexity(100)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:87: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


[2094.574736957681]
[2094.574736957681, 2059.525552303522]
[2094.574736957681, 2059.525552303522, 2043.9965057438906]
[2094.574736957681, 2059.525552303522, 2043.9965057438906, 2037.8941297625422]
[2094.574736957681, 2059.525552303522, 2043.9965057438906, 2037.8941297625422, 2031.6604770995425]
[2094.574736957681, 2059.525552303522, 2043.9965057438906, 2037.8941297625422, 2031.6604770995425, 2029.2605681724028]
[2094.574736957681, 2059.525552303522, 2043.9965057438906, 2037.8941297625422, 2031.6604770995425, 2029.2605681724028, 2026.3726286490512]
[2094.574736957681, 2059.525552303522, 2043.9965057438906, 2037.8941297625422, 2031.6604770995425, 2029.2605681724028, 2026.3726286490512, 2021.071645210049]
[2094.574736957681, 2059.525552303522, 2043.9965057438906, 2037.8941297625422, 2031.6604770995425, 2029.2605681724028, 2026.3726286490512, 2021.071645210049, 2020.4832696317546]
[2094.574736957681, 2059.525552303522, 2043.9965057438906, 2037.8941297625422, 2031.6604770995425, 2029.260568